In [1]:
import torch
from torch import nn, load, tensor
import os
device = 'mps'
checkpoint = load("/Users/jonathanmiddleton/models/my_checkpoints/state_step005960_350M.pt", map_location='mps')

In [2]:
from model import GPT, Hyperparameters as args
model: nn.Module = GPT(vocab_size=args.vocab_size, num_layers=16, num_heads=8, model_dim=1024,
                       max_seq_len=max(args.train_seq_len, args.val_seq_len), device='mps')
state_dict = checkpoint['model']

In [3]:
unwanted_prefix = '_orig_mod.'
for k,v in list(state_dict.items()):
    if k.startswith(unwanted_prefix):
        state_dict[k[len(unwanted_prefix):]] = state_dict.pop(k)
model.load_state_dict(state_dict)


<All keys matched successfully>

In [4]:
model.eval()
model.to('mps')
model = torch.compile(model)

In [5]:
import tiktoken
enc = tiktoken.get_encoding("gpt2")
encode = lambda s: enc.encode(s, allowed_special={"<|endoftext|>"})
decode = lambda l: enc.decode(l)

In [6]:
start_ids = encode("My name is Jonathan ")
x = tensor(start_ids, dtype=torch.long, device='mps')[None, ...]

In [7]:
ctx = torch.amp.autocast(device_type='mps', dtype=torch.bfloat16)
with torch.no_grad():
    with ctx:
        y = model.generate(x, max_new_tokens=100, temperature=0.7)
        print(decode(y[0].tolist()))

TypeError: GPT.forward() missing 2 required positional arguments: 'target_seq' and 'sliding_window_num_blocks'